# Test ModSec Analyzer

Notebook pour tester le module `modsec_analyzer`.

In [23]:
# Ajouter le package au path
import sys
sys.path.insert(0, "packages/modsec_analyzer/src")

# Autoreload pour recharger les modules modifiés
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Imports
from modsec_analyzer.parsing import parse_file
from modsec_analyzer.domain import (
    Rule, SecRule, SecAction, Directive,
    Variable, Operator, Action, Argument,
    RuleSet
)
from collections import Counter

In [25]:
# Chemin vers un fichier de configuration CRS
# Modifier ce chemin selon votre installation
CRS_PATH = "./experiments/config/modsecurity/setup.conf"
# ou un fichier de règles spécifique
# CRS_PATH = "/path/to/coreruleset/rules/REQUEST-942-APPLICATION-ATTACK-SQLI.conf"

## Test parsing avec résolution des includes

In [26]:
# Parser avec résolution des includes (peut prendre quelques secondes)
ruleset_full = parse_file(CRS_PATH, resolve_includes=True)

print(f"Nombre total de rules: {len(ruleset_full.rules)}")

Nombre total de rules: 730


In [27]:
# Fichiers sources uniques
unique_files = set(rule.file_path for rule in ruleset_full.rules)
print(f"\nFichiers parsés ({len(unique_files)}):")
for f in sorted(unique_files):
    print(f"  {f}")


Fichiers parsés (28):
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs-setup.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-901-INITIALIZATION.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-905-COMMON-EXCEPTIONS.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-911-METHOD-ENFORCEMENT.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-913-SCANNER-DETECTION.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-920-PROTOCOL-ENFORCEMENT.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-921-PROTOCOL-ATTACK.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-922-MULTIPART-ATTACK.conf
  /home/lucas/reforge/WAF-GUARD/experiments/config/modsecurity/crs_rules/REQUEST-930-APPLICATION-ATTACK-LFI.conf
  /home/lucas/reforge/WAF-GUARD/

In [32]:
print(ruleset_full.filter(filename="crs_rules"))

RuleSet (715 rules from 26 file(s))
  Types:
    SecRule: 677
    SecMarker: 30
    SecAction: 7
    SecComponentSignature: 1

  Tags:
    attack:
      rce: 90
      protocol: 74
      sqli: 60
      xss: 33
      disclosure: 33
      injection-php: 20
      injection-generic: 7
      multipart-header: 6
      lfi: 5
      rfi: 5
      generic: 3
      ssrf: 3
      fixation: 3
      reputation-scanner: 1
      deprecated-header: 1
      ssti: 1
      injection-java: 1
      (no tag): 347
    application:
      multi: 308
      (no tag): 376
    language:
      multi: 218
      php: 52
      shell: 33
      java: 15
      javascript: 4
      ruby: 3
      aspnet: 2
      ldap: 1
      powershell: 1
      perl: 1
      (no tag): 355
    platform:
      multi: 254
      unix: 27
      windows: 13
      internet-explorer: 13
      iis: 6
      nodejs: 5
      apache: 3
      tomcat: 1
      msaccess: 1
      oracle: 1
      db2: 1
      emc: 1
      firebird: 1
      frontbase: 1
      h

In [33]:
counter=0
for i in range(1, 6):
    rules = ruleset_full.filter(phase=i)
    print(f"\nRègles pour la phase {i} : {len(rules)}")
    counter += len(rules)
print(f"\nTotal des règles phasées: {counter}")


Règles pour la phase 1 : 173

Règles pour la phase 2 : 280

Règles pour la phase 3 : 43

Règles pour la phase 4 : 109

Règles pour la phase 5 : 13

Total des règles phasées: 618


In [30]:
rules=ruleset_full.filter(pl=1)
print(f"\nRègles pour le niveau de paranoïa 1 : {len(rules)}")
rules=ruleset_full.filter(pl=2)
print(f"\nRègles pour le niveau de paranoïa 2 : {len(rules)}")
rules=ruleset_full.filter(pl=3)
print(f"\nRègles pour le niveau de paranoïa 3 : {len(rules)}")
rules=ruleset_full.filter(pl=4)
print(f"\nRègles pour le niveau de paranoïa 4 : {len(rules)}")


Règles pour le niveau de paranoïa 1 : 604

Règles pour le niveau de paranoïa 2 : 692

Règles pour le niveau de paranoïa 3 : 721

Règles pour le niveau de paranoïa 4 : 730


In [31]:
rules=ruleset_full.filter(action="chain")
print(rules)

RuleSet (70 rules from 13 file(s))
  Types:
    SecRule: 70

  Tags:
    attack:
      protocol: 28
      rce: 8
      sqli: 4
      generic: 2
      rfi: 2
      fixation: 2
      multipart-header: 1
      xss: 1
      disclosure: 1
      (no tag): 21
    application:
      multi: 48
      (no tag): 22
    language:
      multi: 45
      java: 2
      aspnet: 1
      shell: 1
      (no tag): 21
    platform:
      multi: 43
      apache: 2
      windows: 2
      unix: 1
      tomcat: 1
      iis: 1
      (no tag): 21


## Analyse visuelle — Impact du Paranoia Level

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# CRS rules only
crs = ruleset_full.filter(filename="crs_rules")

# Cumulative rulesets per PL (filter(pl=N) returns rules with PL <= N)
pl_rulesets = {pl: crs.filter(pl=pl) for pl in range(1, 5)}

# Severity order for consistent plotting
SEVERITY_ORDER = ["CRITICAL", "ERROR", "WARNING", "NOTICE", "(none)"]
SEVERITY_COLORS = {
    "CRITICAL": "#d32f2f",
    "ERROR": "#f57c00",
    "WARNING": "#fbc02d",
    "NOTICE": "#64b5f6",
    "(none)": "#bdbdbd",
}

In [ ]:
# Plot 4 — Severity Distribution globale
sev_dist = crs.severity_distribution
sev_labels = [s for s in SEVERITY_ORDER if sev_dist.get(s, 0) > 0]
sev_values = [sev_dist[s] for s in sev_labels]
sev_colors = [SEVERITY_COLORS[s] for s in sev_labels]

fig, ax = plt.subplots(figsize=(8, 3))
bars = ax.barh(sev_labels, sev_values, color=sev_colors)
ax.bar_label(bars, fmt="%d", padding=4)
ax.set_xlabel("Nombre d'action rules")
ax.set_title("Distribution des sévérités — CRS Rules")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Plot 1 — PL Delta × Severity (stacked bar)
# Compute severity deltas: what each PL level ADDS over the previous
pl_cumulative_sev = {pl: pl_rulesets[pl].severity_distribution for pl in range(1, 5)}

delta_data = {}
delta_data["PL1"] = {s: pl_cumulative_sev[1].get(s, 0) for s in SEVERITY_ORDER}
for pl in range(2, 5):
    delta_data[f"PL{pl} only"] = {
        s: pl_cumulative_sev[pl].get(s, 0) - pl_cumulative_sev[pl - 1].get(s, 0)
        for s in SEVERITY_ORDER
    }

df_delta = pd.DataFrame(delta_data).T
# Keep only columns with data
df_delta = df_delta[[s for s in SEVERITY_ORDER if df_delta[s].sum() > 0]]

fig, ax = plt.subplots(figsize=(9, 5))
df_delta.plot(
    kind="bar",
    stacked=True,
    color=[SEVERITY_COLORS[s] for s in df_delta.columns],
    ax=ax,
    edgecolor="white",
    linewidth=0.5,
)

# Add total label on top of each bar
for i, pl_label in enumerate(df_delta.index):
    total = df_delta.loc[pl_label].sum()
    ax.text(i, total + 2, str(int(total)), ha="center", va="bottom", fontweight="bold")

ax.set_ylabel("Nombre d'action rules ajoutées")
ax.set_title("Rules ajoutées par Paranoia Level — par sévérité")
ax.set_xticklabels(df_delta.index, rotation=0)
ax.legend(title="Severity", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Plot 2 — Heatmap Attack Coverage × PL (cumulative)
# Build matrix: rows = attack sub-types, columns = PL levels
attack_types = [
    t for t, _ in crs.tag_distribution("attack").most_common()
    if t != "(untagged)"
]

heatmap_data = {}
for pl in range(1, 5):
    dist = pl_rulesets[pl].tag_distribution("attack")
    heatmap_data[f"PL{pl}"] = {t: dist.get(t, 0) for t in attack_types}

df_heatmap = pd.DataFrame(heatmap_data, index=attack_types)

fig, ax = plt.subplots(figsize=(6, max(5, len(attack_types) * 0.4)))
sns.heatmap(
    df_heatmap,
    annot=True,
    fmt="d",
    cmap="YlOrRd",
    linewidths=0.5,
    ax=ax,
    cbar_kws={"label": "Nombre de rules"},
)
ax.set_title("Couverture par type d'attaque × Paranoia Level")
ax.set_ylabel("Type d'attaque")
ax.set_xlabel("Paranoia Level (cumulatif)")
plt.tight_layout()
plt.show()

In [ ]:
# Plot 3 — Heatmap normalisée (% du PL4)
# Each row normalized to 100% = PL4 count for that attack type
df_normalized = df_heatmap.div(df_heatmap["PL4"], axis=0) * 100
# Replace NaN (0/0 for types with 0 rules at PL4) with 0
df_normalized = df_normalized.fillna(0)

fig, ax = plt.subplots(figsize=(6, max(5, len(attack_types) * 0.4)))
sns.heatmap(
    df_normalized,
    annot=True,
    fmt=".0f",
    cmap="RdYlGn",
    vmin=0,
    vmax=100,
    linewidths=0.5,
    ax=ax,
    cbar_kws={"label": "% de couverture (PL4 = 100%)"},
)
ax.set_title("% de couverture atteint par Paranoia Level (PL4 = 100%)")
ax.set_ylabel("Type d'attaque")
ax.set_xlabel("Paranoia Level (cumulatif)")
plt.tight_layout()
plt.show()